## 1. Download pretrained word2vec from malaya

In [1]:
!wget https://s3-ap-southeast-1.amazonaws.com/huseinhouse-storage/v13/word2vec/word2vec-wiki-nce-256.p

--2019-05-11 18:41:18--  https://s3-ap-southeast-1.amazonaws.com/huseinhouse-storage/v13/word2vec/word2vec-wiki-nce-256.p
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.32.81
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.32.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 726360284 (693M) [application/x-www-form-urlencoded]
Saving to: ‘word2vec-wiki-nce-256.p’

word2vec-wiki-nce-2 100%[===================>] 692.71M  3.60MB/s    in 3m 11s  

2019-05-11 18:44:29 (3.62 MB/s) - ‘word2vec-wiki-nce-256.p’ saved [726360284/726360284]



In [2]:
import pickle
import tensorflow as tf

In [3]:
with open('word2vec-wiki-nce-256.p', 'rb') as fopen:
    word2vec = pickle.load(fopen)

In [4]:
word2vec.keys()

dict_keys(['rev_dictionary', 'nce_weights', 'dictionary'])

In [5]:
word2vec['dictionary']['ayam']

1187

In [13]:
vectors = word2vec['nce_weights']
dictionary = word2vec['dictionary']
rev_dictionary = word2vec['rev_dictionary']

In [14]:
class Model:
    def __init__(self):
        self._embedding = tf.convert_to_tensor(vectors, dtype = tf.float32)
        self.X = tf.placeholder(
                tf.float32, [None, vectors.shape[1]]
        )
        normed_embedding = tf.nn.l2_normalize(self._embedding, axis = 1)
        normed_array = tf.nn.l2_normalize(self.X, axis = 1)
        self.cosine_similarity = tf.matmul(
            normed_array, tf.transpose(normed_embedding, [1, 0])
        )

In [15]:
tf.reset_default_graph()
model = Model()
sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [25]:
string_positive = 'kerajaan sebenarnya sangat prihatin dengan rakyatnya dapat diskaun dan segalanya'
string_negative = 'minyak naik lagi harga klau gaji naik juga takpe juga'

In [12]:
import random

In [16]:
def augmentation(string, threshold = 0.5, count = 5, k = 8):
    string = string.split()
    selected = []
    while not len(selected):
        selected = [(no, w) for no, w in enumerate(string) if random.random() > threshold]
    indices, words = [i[0] for i in selected], [i[1] for i in selected]
    
    batches = vectors[[dictionary[w] for w in words]]
    top_k = tf.nn.top_k(model.cosine_similarity, k = k)
    results = sess.run(top_k, feed_dict = {model.X: batches})
    words = []
    for result in results.indices:
        words.append([rev_dictionary[i] for i in result])
    augmented = []
    for i in range(count):
        string_ = string[:]
        for no in range(len(words)):
            index = random.randint(0, len(words[no]) - 1)
            string_[indices[no]] = words[no][index]
        augmented.append(' '.join(string_))
    return augmented   

In [21]:
%%time
augmentation(string_positive)

CPU times: user 8.54 s, sys: 692 ms, total: 9.23 s
Wall time: 9.28 s


['pemerintah sememangnya sangat prihatin kerana warganya dapat sewaan hingga nasibnya',
 'pemerintah ternyata sangat prihatin untuk kaumnya dapat keselanjarannya serta segalanya',
 'perlembagaan sebenarnya sangat prihatin selepas penyokongnya dapat sewaan dan laguku',
 'kabinet sebenarnya sangat prihatin dalam penduduknya dapat taksiran dan nasibnya',
 'kerajaan memang sangat prihatin dengan orangnya dapat milodon tetapi nasibnya']

## Malaya implementation

You can check stable text augmentation implementation at https://malaya.readthedocs.io/en/latest/Generator.html